In [2]:
import os

import cv2

DATA_DIR = r'F:\tensorls\sign language CNN 2\images'

number_of_classes = 3
dataset_size = 100

cap = cv2.VideoCapture(0)
for j in range(number_of_classes):
    if not os.path.exists(os.path.join(DATA_DIR, str(j))):
        os.makedirs(os.path.join(DATA_DIR, str(j)))

    print('Collecting data for class {}'.format(j))

    done = False
    while True:
        ret, frame = cap.read()
        cv2.putText(frame, 'Ready? Press "Q" ! :)', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3,
                    cv2.LINE_AA)
        cv2.imshow('frame', frame)
        if cv2.waitKey(25) == ord('q'):
            break

    counter = 0
    while counter < dataset_size:
        ret, frame = cap.read()
        cv2.imshow('frame', frame)
        cv2.waitKey(25)
        cv2.imwrite(os.path.join(DATA_DIR, str(j), '{}.jpg'.format(counter)), frame)

        counter += 1

cap.release()
cv2.destroyAllWindows()

In [2]:
import os 
import mediapipe as mp
import matplotlib.pyplot as plt
import cv2 as cv
import pickle

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles


hands = mp_hands.Hands(static_image_mode=True,min_detection_confidence=0.3)


land marks on hands 

In [ ]:
DATA_DIR = r'F:\tensorls\sign language using random forest & test CNN\images'

for dir_ in os.listdir(DATA_DIR):
    for img_path in os.listdir(os.path.join(DATA_DIR,dir_))[:1]:
        img = cv.imread(os.path.join(DATA_DIR,dir_,img_path))
        img_rgb = cv.cvtColor(img,cv.COLOR_BGR2RGB)

        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                 mp_drawing.draw_landmarks(img_rgb,hand_landmarks,mp_hands.HAND_CONNECTIONS,
                                      
                                      mp_drawing_styles.get_default_hand_landmarks_style(),
                                      mp_drawing_styles.get_default_hand_connections_style()
                                      
                                      )

        plt.figure()
        plt.imshow(img_rgb)
plt.show()  


Arrays of lands marks needed

then saved into a file 

In [4]:
DATA_DIR = r'F:\tensorls\sign language using random forest & test CNN\images'

data=[]
labels=[]

for dir_ in os.listdir(DATA_DIR):
    for img_path in os.listdir(os.path.join(DATA_DIR,dir_)):
        img = cv.imread(os.path.join(DATA_DIR,dir_,img_path))
        img_rgb = cv.cvtColor(img,cv.COLOR_BGR2RGB)

        data_aux=[]
        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                 for i in range(len(hand_landmarks.landmark)):
                     x= hand_landmarks.landmark[i].x
                     y = hand_landmarks.landmark[i].y

                     data_aux.append(x)
                     data_aux.append(y)

            data.append(data_aux)
            labels.append(dir_)

f = open('data1.pickle','wb')
pickle.dump({'data':data,'labels':labels},f)
f.close()

using a  CNN 

In [4]:
import tensorflow 
import numpy as np
from keras.layers import Conv2D, Dropout,MaxPooling2D,Dense,GlobalAveragePooling2D,Flatten
from keras.models import Sequential
from keras.losses import CategoricalCrossentropy

from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping


In [5]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split


with open('data.pickle', 'rb') as f:
    data_dict = pickle.load(f)

data = data_dict['data']
labels = data_dict['labels']

X = np.array(data)
y = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)


model = Sequential([
    Dense(128, input_dim=X.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(set(y)), activation='softmax')  
])





In [6]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [7]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20
8/8 [==============================] - 2s 78ms/step - loss: 1.0608 - accuracy: 0.4667 - val_loss: 1.0210 - val_accuracy: 0.6833
Epoch 2/20
8/8 [==============================] - 0s 14ms/step - loss: 0.9958 - accuracy: 0.7000 - val_loss: 0.9574 - val_accuracy: 1.0000
Epoch 3/20
8/8 [==============================] - 0s 14ms/step - loss: 0.9299 - accuracy: 1.0000 - val_loss: 0.8913 - val_accuracy: 1.0000
Epoch 4/20
8/8 [==============================] - 0s 13ms/step - loss: 0.8643 - accuracy: 1.0000 - val_loss: 0.8178 - val_accuracy: 1.0000
Epoch 5/20
8/8 [==============================] - 0s 14ms/step - loss: 0.7811 - accuracy: 1.0000 - val_loss: 0.7155 - val_accuracy: 1.0000
Epoch 6/20
8/8 [==============================] - 0s 14ms/step - loss: 0.6922 - accuracy: 1.0000 - val_loss: 0.6340 - val_accuracy: 1.0000
Epoch 7/20
8/8 [==============================] - 0s 13ms/step - loss: 0.6008 - accuracy: 1.0000 - val_loss: 0.5302 - val_accuracy: 1.0000
Epoch 8/20
8/8 [===========

In [8]:
f = open('sequenece_model.p','wb')
pickle.dump({'model':model},f)
f.close()

using random forest

In [4]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [5]:
data_dict = pickle.load(open('data1.pickle','rb'))

data = np.asarray(data_dict['data'])
labels = np.asarray(data_dict['labels'])



Training the model and saving it 

In [7]:
X_train,X_test,y_train,y_test = train_test_split(data,labels,test_size=0.2,shuffle=True,stratify=labels)

model  = RandomForestClassifier()

model.fit(X_train,y_train)

y_predict = model.predict(X_test)


score = accuracy_score(y_predict,y_test)

score

f = open('model1.p','wb')
pickle.dump({'model':model},f)
f.close()

testing the model Random Forest

In [6]:
import cv2 as cv

In [7]:
model_dict = pickle.load(open('model1.p','rb'))
model = model_dict['model']


labels_dict = {0:'Hello',1:'Peace',2:'Rock'}

cap = cv.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles


hands = mp_hands.Hands(static_image_mode=True,min_detection_confidence=0.3)


while True:
    data_aux=[]
    x_ = []
    y_ = []

    ret,frame = cap.read()

    frame_rgb = cv.cvtColor(frame,cv.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                 
                 mp_drawing.draw_landmarks(frame,hand_landmarks,mp_hands.HAND_CONNECTIONS,
                                      
                                      mp_drawing_styles.get_default_hand_landmarks_style(),
                                      mp_drawing_styles.get_default_hand_connections_style()
                                      
                 )
                 
            for hand_landmarks in results.multi_hand_landmarks:
                 for i in range(len(hand_landmarks.landmark)):
                     x= hand_landmarks.landmark[i].x
                     y = hand_landmarks.landmark[i].y

                     data_aux.append(x)
                     data_aux.append(y)
                     x_.append(x)
                     y_.append(y)

            H,W,_ = frame.shape

            x1 = int(min(x_)*W)          
            y1 = int(min(y_)*H)

            x2 = int(max(x_)*W)
            y2 = int(max(y_) *H)     

            prediction = model.predict([np.asarray(data_aux)])
            predicted_character = labels_dict[int(prediction[0])]

            cv.rectangle(frame,(x1,y1),(x2,y2),(0,0,0),4)

            cv.putText(frame,predicted_character,(x1,y1),cv.FONT_HERSHEY_COMPLEX,1.3,(0,128,0),3)
                               

    
    cv.imshow('frame',frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()



CNN model test

In [ ]:
model_dict = pickle.load(open('sequenece_model.p','rb'))
model = model_dict['model']


labels_dict = {0:'A',1:'B',2:'L'}

cap = cv.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles


hands = mp_hands.Hands(static_image_mode=True,min_detection_confidence=0.3)


while True:
    data_aux=[]
    x_ = []
    y_ = []

    ret,frame = cap.read()

    frame_rgb = cv.cvtColor(frame,cv.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                 
                 mp_drawing.draw_landmarks(frame,hand_landmarks,mp_hands.HAND_CONNECTIONS,
                                      
                                      mp_drawing_styles.get_default_hand_landmarks_style(),
                                      mp_drawing_styles.get_default_hand_connections_style()
                                      
                 )
                 
            for hand_landmarks in results.multi_hand_landmarks:
                 for i in range(len(hand_landmarks.landmark)):
                     x= hand_landmarks.landmark[i].x
                     y = hand_landmarks.landmark[i].y

                     data_aux.append(x)
                     data_aux.append(y)
                     x_.append(x)
                     y_.append(y)

            H,W,_ = frame.shape

            x1 = int(min(x_)*W)          
            y1 = int(min(y_)*H)

            x2 = int(max(x_)*W)
            y2 = int(max(y_) *H)     
            data_aux = np.array(data_aux).reshape(1, -1)

            prediction = model.predict(data_aux)

            predicted_classes = np.argmax(prediction, axis=1)
            predicted_characters = [labels_dict[class_idx] for class_idx in predicted_classes]
            predicted_characters_str = ', '.join(predicted_characters)



            cv.rectangle(frame,(x1,y1),(x2,y2),(0,0,0),4)

            cv.putText(frame, predicted_characters_str, (x1, y1), cv.FONT_HERSHEY_COMPLEX, 1.3, (0, 0, 0), 3)

                               

    
    cv.imshow('frame',frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

